# i-TED spectrum conversion

In [1]:
pkg_ver = lambda pkg: "{:<20}{:}".format(pkg.__name__,pkg.__version__)

# ROOT
import uproot
print(pkg_ver(uproot))
import ROOT

# Machine Learning
import sklearn
print(pkg_ver(sklearn))
import torch
print(pkg_ver(torch))

# Data science
import scipy
print(pkg_ver(scipy))
import numpy
print(pkg_ver(numpy))
import pandas
print(pkg_ver(pandas))

# Visualizations
import matplotlib
print(pkg_ver(matplotlib))
import matplotlib.pyplot as plt

import tqdm
print(pkg_ver(tqdm))

import glob

uproot              4.3.5
Welcome to JupyROOT 6.28/04
sklearn             1.2.2
torch               2.0.1
scipy               1.10.1
numpy               1.24.3
pandas              1.5.3
matplotlib          3.7.1
tqdm                4.65.0


In [15]:
list_files = [
    "/run/media/bgameiro/d043b5e4-57a4-457e-8839-cb1adc9c72bc/Data/CMAM2023/data_2023_06_12/run1_calEu_D.2023_06_12_T.19_49_58_C.itedE_2023.05.30_4.0v_887C_60s_singleThread_Thr0.root",
    #"/run/media/bgameiro/d043b5e4-57a4-457e-8839-cb1adc9c72bc/Data/CMAM2023/data_2023_06_13/Run08_BeamOn_BeTarget_4MeV_55nA_D.2023_06_13_T.15_18_38_C.itedE_CMAM_4.0v_887C_1800s_singleThread_Thr0.root"
    "/run/media/bgameiro/d043b5e4-57a4-457e-8839-cb1adc9c72bc/Data/CMAM2023/data_2023_06_13/Run11_Beamoff_Eu152_calibration_D.2023_06_13_T.18_09_32_C.itedE_CMAM_4.0v_887C_180s_singleThread_Thr0.root"
]   

In [10]:
iTED="E"
dir_ = f"/run/media/bgameiro/d043b5e4-57a4-457e-8839-cb1adc9c72bc/Data/CMAM2023/Calibration/{iTED}/"

for f in list_files:
    
    name = f.split("/")[-1][:-5]
    
    cw = name[-3:]
    
    tfile = uproot.open(f)
    
    S = tfile[f"SCATTERER_{iTED}_amplitude_spectra;1"]
    pandas.DataFrame(S.values() , columns=["Data"]).rename_axis('Energy').to_csv(f'{dir_}{iTED}0/88C_CMAM/{name}.csv')
    
    A1= tfile[f"ABSORBER_{iTED}_1_amplitude_spectra;1"]
    pandas.DataFrame(A1.values(), columns=["Data"]).rename_axis('Energy').to_csv(f'{dir_}{iTED}1/88C_CMAM/{name}.csv')
    
    A2= tfile[f"ABSORBER_{iTED}_2_amplitude_spectra;1"]
    pandas.DataFrame(A2.values(), columns=["Data"]).rename_axis('Energy').to_csv(f'{dir_}{iTED}2/88C_CMAM/{name}.csv')
    
    A3= tfile[f"ABSORBER_{iTED}_3_amplitude_spectra;1"]
    pandas.DataFrame(A3.values(), columns=["Data"]).rename_axis('Energy').to_csv(f'{dir_}{iTED}3/88C_CMAM/{name}.csv')
   
    A4= tfile[f"ABSORBER_{iTED}_4_amplitude_spectra;1"]
    pandas.DataFrame(A4.values(), columns=["Data"]).rename_axis('Energy').to_csv(f'{dir_}{iTED}4/88C_CMAM/{name}.csv')

In [17]:
calp = glob.glob('/run/media/bgameiro/d043b5e4-57a4-457e-8839-cb1adc9c72bc/Data/CMAM2023/Calibration/**/*.CALp', recursive=True)

calibrations = []

for i in calp:
    line = pandas.read_csv(
        i,
        sep = "      :  ",
        skiprows=[0,4,5,6],
        header=None,
        engine="python"
    ).drop([0], axis=1).T
    
    line["crystal"] = i.split("/")[-1][:2]
    line["configuration"] = i.split("/")[-2]
    if i.split("/")[-2] == "88C_IFIC":
        line["temperature_mean"]   = round(uproot.open(list_files[0])[f'T_ASIC_{line["crystal"].values[0][-1]};1'].values()[1].mean(),2)
        line["temperature_min"]    = uproot.open(list_files[0])[f'T_ASIC_{line["crystal"].values[0][-1]};1'].values()[1].min()
        line["temperature_max"]    = uproot.open(list_files[0])[f'T_ASIC_{line["crystal"].values[0][-1]};1'].values()[1].max()
        line["temperature_change"] = round(line["temperature_max"]-line["temperature_min"],1)
    elif i.split("/")[-2] == "88C_CMAM":
        line["temperature_mean"]   = round(uproot.open(list_files[1])[f'T_ASIC_{line["crystal"].values[0][-1]};1'].values()[1].mean(),2)
        line["temperature_min"]    = uproot.open(list_files[1])[f'T_ASIC_{line["crystal"].values[0][-1]};1'].values()[1].min()
        line["temperature_max"]    = uproot.open(list_files[1])[f'T_ASIC_{line["crystal"].values[0][-1]};1'].values()[1].max()
        line["temperature_change"] = round(line["temperature_max"]-line["temperature_min"],1)
    
    
    calibrations.append(line)

In [18]:
pandas.concat(calibrations,ignore_index=True).sort_values(by=['crystal','configuration'])

,0,1,2,crystal,configuration,temperature_mean,temperature_min,temperature_max,temperature_change
6,115.4820,2.343750,0.000473,E0,88C_CMAM,24.61,24.6,24.8,0.2
7,87.1183,1.312900,0.001402,E0,88C_IFIC,24.34,24.3,24.4,0.1
0,181.9450,2.287200,0.000473,E1,88C_CMAM,24.83,24.8,25.0,0.2
1,252.1430,0.932148,0.001726,E1,88C_IFIC,24.60,24.6,24.6,0.0
8,161.2300,2.710730,0.000492,E2,88C_CMAM,24.99,24.8,25.0,0.2
9,121.9170,1.649660,0.001643,E2,88C_IFIC,24.72,24.6,24.8,0.2
4,221.7720,2.315470,0.000668,E3,88C_CMAM,24.39,24.3,24.4,0.1
5,279.8830,1.150800,0.001748,E3,88C_IFIC,24.09,23.9,24.1,0.2
2,180.8790,3.151310,0.000756,E4,88C_CMAM,25.70,25.7,25.7,0.0
3,155.3700,1.897040,0.002089,E4,88C_IFIC,25.28,25.2,25.3,0.1


In [19]:
pandas.concat(calibrations,ignore_index=True).sort_values(by=['crystal']).to_csv("iTED_E_calibration_CMAM.csv")